In [ ]:
#Associative Learning with all training samples

maxCurrent           = 1e-5
numClasses=3
onAmp = 0.1
signalType='DC'
run_time=1
dt=0.01
num_drain_training=3
num_source_training=9
t=[None]*numClasses
sim=[None]*numClasses
state=[None]*numClasses
numTrainingSamples=8
rest=[0]*9
restDur=10
trainOrder=np.array(range(numTrainingSamples))
np.random.shuffle(trainOrder)

#Run for each class:
for classNum in tqdm(range(numClasses)):
    targetClass   = classNum #'z','v','n'
    target_values = allTargets[targetClass]
    chosenDrain   = drain_pool[targetClass]
    if len(elecDrain) == 3:
        targetClassVals=[0,1,2]
    else:
        targetClassVals=[0,1,2,3]
        target_values=np.hstack((target_values,0)) #add extra 0 for 4th drain that acts just to balance the network

    trainingInputs  = inputs[targets==targetClassVals[targetClass]][:-2]
    testingInputs   = inputs[targets==targetClassVals[targetClass]][-2:]
    
    #expand each value to run the number of timesteps:
    newRow=np.zeros((len(trainingInputs),int(run_time/dt)+restDur,9))
    count=0
    for i in trainingInputs[trainOrder]:
        for j in range(int(run_time/dt)+restDur):
            if j < int(run_time/dt):
                newRow[count][j]=i
            else:
                newRow[count][j]=rest
        count=count+1
    stimulus=np.concatenate(newRow,axis=0).T*onAmp

    newSignal=[]
    for i in range(num_drain_training):
        newSignal.append(stimulus__(biasType='Drain',T=dt*stimulus.shape[1],dt=dt))
    for i in range(len(stimulus)):
        newSignal.append(stimulus__(biasType='Custom',onAmp=onAmp,T=dt*stimulus.shape[1],dt=dt,customSignal=stimulus[i]))
#     set_trace()
    sim[classNum] = getNWState_backprop(newSignal,None,drain_pool,sources,run_time=len(newSignal[0].signal)*dt)
    t[classNum],v=calcOutputs(sim[classNum],sources,drain_pool)
    
    #if we want only one drain, use [chosenDrain] instead of drain_pool
